In [4]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [5]:
v3 = Vector([3, 4, 5])
format(v3)

'(3.0, 4.0, 5.0)'

In [6]:
format(Vector([1, 1]), 'h')

'<1.4142135623730951, 0.7853981633974483>'

In [7]:
format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [8]:
format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [9]:
format(Vector([1, 1, 1]), 'h')

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [10]:
format(Vector([2, 2, 2]), '.3eh')

'<3.464e+00, 9.553e-01, 7.854e-01>'

In [11]:
format(Vector([0, 0, 0]), '0.5fh')

'<0.00000, 0.00000, 0.00000>'

In [12]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [13]:
format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'